In [5]:
from operator import attrgetter
from sortedcontainers import SortedList
import bisect
import math
import random as rand
from pyskiplist import SkipList

In [10]:
def binarysearch(a, x):
    i = bisect.bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return True
    else:
        return False

In [11]:

##FIXED VERSION
class MaxList:
    def __init__(self):
        self.nodes = []
        self.max = float("-inf")

    def __lt__(self, other):
        if isinstance(other, int):
            return self.max < other
        else:
            return self.max < other.max

class MinMaxList:
    def __init__(self):
        self.sublists = []
        self.min = float("inf")
        self.max = float("-inf")

    def __lt__(self, other):
        return self.max > other

class TeleportList:

    def __init__(self):
        self.height = -1
        self.blists = []
        self.load = 999

    def lookup(self, nr):
        ## Search MinMaxLists in TeleportList
        for he in self.blists:
            if nr < he.min:
                break
            else:
                if he.max >= nr:
                    ## Search MaxLists in MinMaxList
                    i = bisect.bisect_left(he.sublists, nr)
                    for j in range(i, len(he.sublists)):
                        if binarysearch(he.sublists[j].nodes, nr):
                            return True
        return False

    def insert(self, nr):
        ## Getting the estimated geometric distribution
        height = int(-(math.log2(random.random())))
        ## Checking whether we need to add new edges
        if self.height < height:
            for i in range(height - self.height):
                self.blists.append(MinMaxList())
                self.blists[self.height + i].sublists.append(MaxList())
            self.height = height

        ## Getting the to-be-added list
        blist = self.blists[height-1]

        ## Doing the search to see which MaxList it should be in
        L = len(blist.sublists)
        i = bisect.bisect_left(blist.sublists, nr)
        ## If it's there don't insert
        if i != L and blist.sublists[i] == nr:
            return False
        else:
            ## If it's smaller than all other elements there then just insort it
            if i == 0:
                new_max = MaxList()
                new_max.nodes.append(nr)
                new_max.max = nr
                blist.sublists.insert(0, new_max)
                #bisect.insort_left(blist.sublists, new_max)
                #blist.sublists.append(new_max)
            ## If it's bigger than just append to end
            elif i == L:
                blist.sublists[-1].nodes.append(nr) ### can be overloaded
                blist.sublists[-1].max = nr
                #if nr > blist.sublists[max(i - 1, 0)].max:
                    #blist.sublists[max(i - 1, 0)].max = nr
                #elem = blist.sublists.pop(-1)
                #elem.max = nr
                #bisect.insort_left(blist.sublists, elem)
                #blist.sublists.sort(key=attrgetter('max'))
            ## Else add it
            else:
                candidate_sublist = blist.sublists[i]
                ## If MaxList has under 1000 elements then just insort
                if len(candidate_sublist.nodes) <= self.load:
                    if nr > candidate_sublist.max:
                    #blist.sublists[max(i - 1, 0)].max = nr
                        elem = blist.sublists.pop(i)
                        bisect.insort_left(elem.nodes, nr)
                        elem.max = nr
                        bisect.insort_left(blist.sublists,elem)
                    else:
                        bisect.insort_left(candidate_sublist.nodes, nr)
                    
                ## Else create a new MaxList and insort it there
                else:
                    new_max = MaxList()
                    new_max.nodes.append(nr)
                    new_max.max = nr
                    #blist.sublists.append(new_max)
                    bisect.insort_left(blist.sublists, new_max)
            ## Update MinMax
            if blist.min > nr:
                blist.min = nr
            if blist.max < nr:
                blist.max = nr
            #blist.sublists.sort(reverse=True, key=attrgetter('max'))################this is slow
            self.blists.sort(key=attrgetter('min'))

    def show_hedges(self):
        for i in self.blists:
            maxes = [j.max for j in i.sublists]
            print(maxes)
            
    def show_edges(self):
        for i in self.blists:
            print("--------" + str(len(i.sublists)) +"-----------")
            for j in i.sublists:
                print(j.nodes)

    def show_minmax(self):
        for i in self.blists:
            print(f'({i.min}, {i.max})')

In [136]:
## FASTER VERSION #in this version we do not keep many half empty maxlists but insstead- fill them fully and then create a new one
class MaxList:
    def __init__(self):
        self.nodes = []
        self.max = float("-inf")

    def __lt__(self, other):
        if isinstance(other, int):
            return self.max < other
        else:
            return self.max < other.max

class MinMaxList:
    def __init__(self):
        self.sublists = []
        self.min = float("inf")
        self.max = float("-inf")
        self.index = 0 #index shows where is the runnnig maxlist 

    def __lt__(self, other):
        if isinstance(other, int):
            return self.max < other
        else:
            return self.max < other.max

class TeleportList:

    def __init__(self):
        self.height = -1
        self.blists = []
        self.load = 999
        self.c = 0 #counter for dynamic load regulating
        

    def lookup(self, nr):
        ## Search MinMaxLists in TeleportList
        k = bisect.bisect_left(self.blists, nr)
        for h in range(k, len(self.blists)):
        #for he in self.blists:
            he = self.blists[h]
            if nr > he.max:
                #print("lohh")
                break
            else:
                #if he.max >= nr:
                    ## Search MaxLists in MinMaxList
                i = bisect.bisect_left(he.sublists, nr)
                
                for j in range(i, len(he.sublists)):
                        #if he.sublists[j].nodes[0] <= nr:
                    if binarysearch(he.sublists[j].nodes, nr):
                        return True
        return False

    def insert(self, nr):
        ## Getting the estimated geometric distribution
        height = int(-(math.log2(random.random())))
        ## Checking whether we need to add new edges
        if self.height < height:
            for i in range(height - self.height):
                self.blists.append(MinMaxList())
                self.blists[self.height + i + 1].sublists.append(MaxList())
            self.height = height

        ## Getting the to-be-added list
        blist = self.blists[height-1]

        ## We know which max list it is because we store the running index in memory
        temp = blist.sublists[blist.index]
        L = len(temp.nodes)
        
        ## If the list is full we create a new one
        if L == self.load + 1:
                new_max = MaxList()
                new_max.nodes.append(nr)
                new_max.max = nr
                i = bisect.bisect_left(blist.sublists, nr)
                blist.index = i
                blist.sublists.insert(i, new_max)
        ## if there is room for more elements, we just add
        else: 
            if nr > temp.max:
                elem = blist.sublists.pop(blist.index)
                elem.nodes.append(nr)
                elem.max = nr
                i = bisect.bisect_left(blist.sublists, elem)
                blist.index = i
                blist.sublists.insert(i, elem)
            else:
                bisect.insort_left(temp.nodes, nr)
                    
        
            ## Update MinMax
        if blist.min > nr:
            blist.min = nr
        if blist.max < nr:
            blist.max = nr
        self.blists.sort(key=attrgetter('min'))
        #we check if we need to upscale the load factor or not (when adding deletion, this needs to have a decreasing 
        #property as well)
        self.c += 1
        if self.c >= 10*self.load:
            self.load *= 5

    def show_hedges(self):
        for i in self.blists:
            maxes = [j.max for j in i.sublists]
            print(maxes)
            
    def show_edges(self):
        for i in self.blists:
            print("-------------------")
            for j in i.sublists:
                print(j.nodes)

    def show_minmax(self):
        for i in self.blists:
            print(f'({i.min}, {i.max})')

In [137]:

hgraph = TeleportList()
skeepFast = SkipList()
slist = SortedList()

nr = 100000

ten_thousand_integers = [rand.randint(1, 1000000) for i in range(nr)]

def ten_k_hg(HG):
    for i in range(nr):
        HG.insert(ten_thousand_integers[i])

def ten_k_sl(sl):
    for i in range(nr):
        sl.add(ten_thousand_integers[i])

def ten_k_skeep(skeep):
    for i in range(nr):
        skeep.insert(ten_thousand_integers[i], ten_thousand_integers[i])

%timeit -r 1 -n 1 ten_k_hg(hgraph)
%timeit -r 1 -n 1 ten_k_sl(slist)
%timeit -r 1 -n 1 ten_k_skeep(skeepFast)

1 loop, best of 1: 514 ms per loop
1 loop, best of 1: 313 ms per loop
1 loop, best of 1: 2.57 s per loop


In [138]:
#nr = 100000
#ten_thousand_integers = [rand.randint(1, 1000000) for i in range(nr)]

def ten_k_hg(HG):
    for i in range(nr):
        HG.lookup(ten_thousand_integers[i])

def ten_k_sl(sl):
    for i in range(nr):
        ten_thousand_integers[i] in sl

def ten_k_skeep(skeep):
    for i in range(nr):
        ten_thousand_integers[i] in skeep

%timeit -r 1 -n 1 ten_k_hg(hgraph) #it is less stable then, e.g if there are many mismatches, the time will double
%timeit -r 1 -n 1 ten_k_sl(slist)
%timeit -r 1 -n 1 ten_k_skeep(skeepFast)

1 loop, best of 1: 2.27 s per loop
1 loop, best of 1: 181 ms per loop
1 loop, best of 1: 1.16 s per loop


In [123]:

hgraph.show_minmax()
hgraph.show_hedges()
print("-------------------")
#hgraph.show_edges()

(34, 999466)
(70, 999691)
(41, 999707)
(635, 999711)
(116, 999723)
(98, 999751)
(418, 999771)
(1339, 999800)
(47, 999859)
(271, 999905)
(384, 999925)
(1188, 999928)
(837, 999969)
(30, 999970)
(401, 999974)
(584, 999974)
(33, 999990)
[999466]
[999691]
[999707]
[999711]
[999723]
[999751]
[999771]
[999800]
[999859]
[999259, 999497, 999905]
[999925]
[999928]
[999969]
[999076, 999837, 999970]
[999974]
[999974]
[999938, 999946, 999972, 999990]
-------------------
